In [1]:
import torch as th
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO
import numpy as np
from Simulator.gui.sim import Simulator

from stable_baselines3.common.env_checker import check_env


import os
log_dir = "log/gym/"
os.makedirs(log_dir, exist_ok=True)

class CustomEnv(gym.Env):
    """Custom Environment that follows gym interface."""

    metadata = {"render_modes": ["human"], "render_fps": 30}

    def __init__(self, render_mode=None):
        super().__init__()
        # Define action and observation space
        # They must be gym.spaces objects
        self.sim_env = Simulator()
        # Example when using discrete actions:
        self.action_space = spaces.Discrete(6)
        # Example for using image as input (channel-first; channel-last also works):
        self.observation_space = spaces.Box(low=-2000, high=2000,
                                            shape=(16,), dtype=np.float64)
        
        assert render_mode is None or render_mode in self.metadata["render_modes"]
        self.render_mode = render_mode
        if render_mode is not None:
            self.sim_env.init_window()

    def step(self, action):
        observation, reward, terminated, truncated, info = self.sim_env.step(action)
        return observation, reward, terminated, truncated, info

    def reset(self, seed=None, options=None):
        observation, info = self.sim_env.reset()
        return observation, info

    def render(self):
        self.sim_env.render()

    def close(self):
        self.sim_env.kill_window()

# env = gym.make("CartPole-v1", render_mode="human")
# env = CustomEnv()

# It will check your custom environment and output additional warnings if needed
# check_env(env)



In [3]:
env = CustomEnv()
policy_kwargs = dict(activation_fn=th.nn.ReLU, net_arch=dict(pi=[1024, 1024, 1024, 1024, 1024, 1024], vf=[1024, 1024, 1024, 1024, 1024, 1024]))
model = PPO("MlpPolicy", env, verbose=1, policy_kwargs=policy_kwargs, device="cuda")
# model.load("Test_ppo_model", env)
model.learn(total_timesteps=500)

model.save("Test_ppo_model")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 192      |
|    ep_rew_mean     | -872     |
| time/              |          |
|    fps             | 132      |
|    iterations      | 1        |
|    time_elapsed    | 15       |
|    total_timesteps | 2048     |
---------------------------------


In [6]:
from stable_baselines3.common import results_plotter

# Helper from the library
results_plotter.plot_results(
    [log_dir], 1e5, results_plotter.X_TIMESTEPS, "PPO"
)


IndexError: index -1 is out of bounds for axis 0 with size 0

<Figure size 800x200 with 0 Axes>

In [9]:
env = CustomEnv(render_mode="human")

model = PPO("MlpPolicy", env, verbose=1, device="cuda")
model.load("Test_ppo_model", env)

vec_env = model.get_env()
obs = vec_env.reset()
while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = vec_env.step(action)
    vec_env.render()
    # VecEnv resets automatically
    # if done:
    #   obs = env.reset()

env.close()

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/alex/.local/lib/python3.10/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


SystemExit: 

/home/alex/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
